<a href="https://colab.research.google.com/github/Babs-Coding/Refonte/blob/main/Exercise_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xgboost
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
#from sklearn.ensemble import GradientBoostingClassifier
#from sklearn.metrics import classification_report, confusion_matrix
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

df = pd.read_csv('Student_data.csv')

df.head()
df.info()
df.describe()
df.isnull().sum()

print("missing values: \n", df.isnull().sum())
print("\nTarget Variable: \n", df['Loan_Status'].value_counts(normalize=True))

num_cols = df.select_dtypes(include=['int64', 'float64']).columns
cat_cols = df.select_dtypes(include='object').columns
imputer = SimpleImputer(strategy='mean')
df[num_cols] = imputer.fit_transform(df[num_cols])

df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

if 'LoanAmount' in df.columns and 'ApplicantIncome' in df.columns: df['Debt'] = df['LoanAmount'] / (df['ApplicantIncome'] + 1)
scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])


X = df.drop([col for col in df.columns if 'Loan_Status' in col], axis=1)
y = df['Loan_Status_Y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

results_df = X_test.copy()
results_df['Predicted_Loan_Status'] = y_pred
results_df['Actual_Loan_Status'] = y_test.values
print(results_df.head())

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

precision = precision_score(y_test, y_pred)
print(f"Precision: {precision}")

recall = recall_score(y_test, y_pred)
print(f"Recall: {recall}")

f1 = f1_score(y_test, y_pred)
print(f"F1 Score: {f1}")

roc_auc = roc_auc_score(y_test, y_pred)
print(f"ROC AUC: {roc_auc}")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB
missing values: 
 Loan_ID               0
Gender               13
Married               3
Dependents           1

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:21:35] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


     ApplicantIncome  ...  Actual_Loan_Status
350         0.602802  ...                True
377        -0.179136  ...                True
163        -0.202563  ...                True
609        -0.410130  ...                True
132        -0.439946  ...                True

[5 rows x 630 columns]
Accuracy: 0.7479674796747967
Precision: 0.7634408602150538
Recall: 0.8875
F1 Score: 0.8208092485549133
ROC AUC: 0.6879360465116279
